### Exploring Masking Algorithm

**Objective:** Mask an specific percentage of words that are label related using whole-word masking

In [1]:
import pandas as pd
import math
import random
import collections
import numpy as np
from typing import Tuple
from ast import literal_eval
import re
import os

In [2]:
# Uncomment this cell if working in collab
from google.colab import drive
drive.mount('/content/drive')

path = "/content/drive/MyDrive/Qbot-gpt/data"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
# Uncomment this line if working locally
# path = "/Users/algiraldoh/Development/qbot-gpt/data"

In [4]:
! pip install transformers datasets scikit-learn huggingface-hub evaluate pynvml transformers[torch] --quiet

### 1. Load the necessary data

In [5]:

# labelled skills
labeled_skills_file = os.path.join(path, "skills_sample.csv")
df_skills = pd.read_csv(labeled_skills_file)
# filter some columns out
df_skills = df_skills.iloc[:,1:3].copy()

#  linkedin data
linkedin_file = os.path.join(path, "jobs_230612.csv")
df_desc = pd.read_csv(linkedin_file, escapechar="\\")

# matched testing data
matches_file = os.path.join(path,'matches.csv')
df_matches = pd.read_csv(matches_file)
df_matches['0_'] = df_matches['0'].apply(lambda x: literal_eval(str(x)))

### 2. Cleansing the data

Starting with the skills column from the skills_data df.

In [6]:
def skills_cleaning(
        df: pd.DataFrame,
        skills_column: str,
        label_column: str,
        re_preparent: re.Pattern=re.compile(r'^([\w\s]*) \('),
        re_parent: re.Pattern=re.compile(r'\(([a-z\s]+)\)'),
        re_hasparent: re.Pattern=re.compile(r'\([a-z\s]+\)'),
) -> pd.DataFrame:
    """
    Function to clean the skills scraped from LinkedIn. The function
    removes the "(programming language)" from the skills. It identifies
    further skills containing parentheses, the splits these in two rows.
    One row with the text before the parentheses and another row with
    text within the parentheses. It also gets rid of duplicate rows and
    rows with missing labels.

    :param df: pandas.DataFrame containing the skills to be cleaned
    :param skills_column: string value of the header of the column with
    the skills
    :param label_column: string value of the header of the column with
    the labels
    :param re_preparent: re.Pattern regex to match text in front of a
    parentheses
    :param re_parent: re.Pattern regex to match text inside a parentheses
    :param re_hasparent: re.Pattern to match if a skill has parentheses
    :return: pandas.DataFrame with the cleaned skills column
    """

    # copying the DF, so we won't overwrite the original
    dfc = df.copy()

    dfc = dfc.dropna(subset=label_column).reset_index(drop=True)

    dfc[skills_column] = dfc[skills_column].str.replace(
        ' (programming language)',
        '',
        regex=False,
    )
    # creating two arrays with the texts from before the parentheses and
    # from within the parentheses
    preparent = dfc[skills_column].str.extract(re_preparent)
    parent = dfc[skills_column].str.extract(re_parent)

    # creating a boolean mask to note which rows have parentheses
    mask = dfc[skills_column].str.contains(re_hasparent, regex=True)

    # if there are parentheses, we create a copy of the parentheses rows
    # and append it to the modified original DF
    if np.any(mask):
        df_new = dfc.loc[mask].copy()
        # the original parentheses rows will have the preparentheses text
        dfc.loc[mask, skills_column] = preparent.loc[mask, 0]
        # the new rows will have the post parentheses text
        df_new.loc[:, skills_column] = parent.loc[mask, 0]

        # appending the new DF to the old, also dropping duplicates
        return pd.concat([dfc, df_new]).drop_duplicates(subset=skills_column)
    else:
        # if no changes are needed, we just drop the duplicates
        return dfc.drop_duplicates(subset=skills_column)

In [7]:
def description_cleaning(
        df: pd.DataFrame,
        desc_column: str,
        re_punc: re.Pattern=re.compile(
            r'([!"#$%\'()*+,./:;<=>?@\]\[\\^_`{|}~])'
        ),
        re_spac: re.Pattern=re.compile(r'[\n\s]+'),
        re_apos: re.Pattern=re.compile(r'\' '),
) -> pd.DataFrame:
    """
    Function to clean the job descriptions, making them ready to be
    "tokenised" by the label creation function. It pads punctuation marks
    with spaces, eliminates redundant white spaces and removes the "about
    the job" header.

    :param df: pandas.DataFrame to clean
    :param desc_column: string value of the column header with the
    description to clean
    :param re_punc: re.Pattern regex to match punctuation marks
    :param re_spac: re.Pattern regex to match white spaces
    :param re_apos: re.Pattern regex to match apostrophes
    :return: pandas.DataFrame with the cleaned job description column
    """

    # copying the DF, so the original gets preserved
    dfc = df.copy()
    dfc[desc_column] = dfc[desc_column].str.lower()
    # padding punctuation marks with white spaces
    dfc[desc_column] = dfc[desc_column].str.replace(
        re_punc,
        r' \1 ', # inserting the first captured group (padded)
        regex=True,
    )
    # removing redundant white spaces
    dfc[desc_column] = dfc[desc_column].str.replace(re_spac, ' ', regex=True)
    # removing the second padding space from apostrophes. so "job ' s"
    # becomes "job 's"
    dfc[desc_column] = dfc[desc_column].str.replace(re_apos, '\'', regex=True)
    dfc[desc_column] = dfc[desc_column].str.replace(
        'about the job',
        '',
        regex=False,
    ).str.strip()

    return dfc

In [8]:
# Cleaning the skills
cleaned_skills_df = skills_cleaning(df=df_skills, skills_column='skills', label_column='label_gen')
# Cleaning job description
cleaned_desc_df = description_cleaning(df=df_desc, desc_column='job_description')



In [9]:
# Additional pre-processing
cleaned_skills_df = cleaned_skills_df.dropna().reset_index(drop=True)

In [10]:
cleaned_skills_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6672 entries, 0 to 6671
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   skills     6672 non-null   object
 1   label_gen  6672 non-null   object
dtypes: object(2)
memory usage: 104.4+ KB


### 3. Labelling

In [11]:
category_dict = {
    'Business': ('B-BUS', 'I-BUS'),
    'Technology': ('B-TECHNOLOGY', 'I-TECHNOLOGY'),
    'Technical': ('B-TECHNICAL', 'I-TECHNICAL'),
    'Soft': ('B-SOFT', 'I-SOFT'),
}

id2label = {
    0: "O",
    1: "B-BUS",
    2: "I-BUS",
    3: "B-TECHNOLOGY",
    4: "I-TECHNOLOGY",
    5: "B-TECHNICAL",
    6: "I-TECHNICAL",
    7: "B-SOFT",
    8: "I-SOFT",
}

label2id = {
    "O": 0,
    "B-BUS": 1,
    "I-BUS": 2,
    "B-TECHNOLOGY": 3,
    "I-TECHNOLOGY": 4,
    "B-TECHNICAL": 5,
    "I-TECHNICAL": 6,
    "B-SOFT": 7,
    "I-SOFT": 8,
}

In [12]:
def make_skills_label_dict(
        df: pd.DataFrame,
        skills_column: str,
        cat_column: str,
        label_dict: dict,
) -> dict:
    """
    Function that takes in a DF with the skills and their categories along with a dictionary with keys corrsponding
    to the skill categories. The function creates a dictionary with the skills as keys and the beginning and
    intermadiate labels as values.

    :param df: pandas.DataFrame with the skills and their categories
    :param skills_column: string value of the column header containing the skills
    :param cat_column: string value of the column header containing the skill category
    :param label_dict: dictionary with the category names as keys and the label tuples as values
    :return: dictionary where the skills are the keys and values are the label tuples based on their category
    """

    skills_label_dict = dict()

    for i,s in enumerate(df[skills_column]):
        value_range = np.arange(len(s.split(" ")))
        bool_array = value_range == 0
        labelled_array = np.where(bool_array, label2id[category_dict[df[cat_column].iloc[i]][0]], label2id[category_dict[df[cat_column].iloc[i]][1]])
        skills_label_dict[s] = list(labelled_array)

    return skills_label_dict

In [13]:

skills_label_dict = make_skills_label_dict(
    df=cleaned_skills_df,
    skills_column='skills',
    cat_column='label_gen',
    label_dict=category_dict,
)


Finding the matches of skills in every description text

In [14]:
def wrap_in_spaces(
    string_val: str,
    re_string: re.Pattern=re.compile(r'([\w\s]+)'),
) -> str:
    """
    Simple function to wrap the input string in spaces.

    :param string_val: string value to wrap
    :param re_string: re.Pattern regex matching word characters and
    spaces
    :return: cleaned string value
    """

    return re.sub(re_string, r' \1 ', string_val)

In [15]:
def get_skills_boolean_matrix(
    df: pd.DataFrame,
    desc_column: str,
    skills_dictionary: dict,
    *args,
    **kwargs,
) -> np.ndarray:
    """
    Function returning a boolean array indicating if the skills found in
    the keys of the input dictionary are in the data frame for each row.
    Therefore, the shape of the output array is
    (df.shape[0], len(skills_dictionary)).

    :param df: pandas.DataFrame to check
    :param desc_column: name of the column to perform the matching on
    :param skills_dictionary: dictionary whose keys will be used for
    matching
    :param args: positional arguments
    :param kwargs: keyword arguments
    :return: numpy.ndarray containing the boolean mask indicating whether
    the skill is present in a row or not
    """

    # creating a container for boolean arrays for each skill
    arrays = []
    # iterating through the skills and checking if they ar present in the
    # descriptions
    for skill_key in list(skills_dictionary.keys()):
        # checking whether the skill is present in each deascription
        array = (
            df[desc_column]
            .str
            .contains(
                # wrapping the skill in spaces, so only relevant matches
                # are picked up
                wrap_in_spaces(
                    string_val=skill_key,
                    *args,
                    **kwargs,
                ),
                regex=False,
            )
        )

        arrays.append(array)

    # we arrange the arrays so there is a row corresponding to each DF row with
    # a boolean corresponding to each skill
    return np.column_stack(arrays)




In [16]:
def label_entry(
        skill: str,
        skills_label_dictionary: dict,
        description_split: list,
        label_list: list,
) -> None:
    """
    Function to detect the position of a skill in a list of strings from the
    description and write the corresponding labels into the labels list -
    corresponding to the description list.

    :param skill: string value of the skill
    :param skills_label_dictionary: dictionary where the keys are the skills and
    values are the associated labels
    :param description_split: list of words from the description (split on white spaces)
    :param label_list: list of labels corresponding to the description_split list (initially all 'O'-s)
    :return: None
    """

    skill_split = skill.split(' ')
    len_ds = len(description_split)
    len_ss = len(skill_split)

    for i in range(len_ds - len_ss + 1):
        if description_split[i: i + len_ss] == skill_split:
            #print(f'{i}, {len_ds=}, {len_ss=}, {len(label_list)=}')
            label_list[i] = skills_label_dictionary[skill][0]
            label_list[i + 1: i + len_ss] = [skills_label_dictionary[skill][1] for _ in range(len_ss - 1)]
        else:
            pass

In [17]:
def label_df(
        df: pd.DataFrame,
        desc_column: str,
        skills_list: list,
        boolean_masks: np.ndarray,
        *args,
        **kwargs,
) -> tuple:
    """
    Function to iterate through a data frame's description column returning two lists, one with the split description
    and one with the corresponding labels.

    :param df: pandas.DataFrame with the description to label
    :param desc_column: string value of the column header
    :param skills_list: list of string values of skills
    :param boolean_masks: numpy.ndarray of boolean values of shape (df.shape[0], len(skills_list)). it corresponds to
    whether a skill is present in a row of the data frame
    :param args: positional arguments
    :param kwargs: keyword arguments
    :return: tuple of lists. one with the split description strings, one with the corresponding label strings
    """

    input_array = []
    label_array = []

    for i in range(df.shape[0]):
        description_split = df[desc_column].values[i].split(' ')
        label_list = [0 for _ in range(len(description_split))]

        for skill in skills_list[boolean_masks[i, :]]:
            label_entry(
                skill=skill,
                description_split=description_split,
                label_list=label_list,
                *args,
                **kwargs,
            )

        input_array.append(description_split)
        label_array.append(label_list)

    return input_array, label_array

In [18]:
re_string = re.compile(r'([\w\s]+)')

bool_masks = get_skills_boolean_matrix(
    df=cleaned_desc_df,
    desc_column='job_description',
    skills_dictionary=skills_label_dict,
    re_string=re_string,
)

bool_masks.shape

(15301, 6672)

In [19]:
skills_all = np.array(list(skills_label_dict.keys()))

input_array, label_array = label_df(
    df=cleaned_desc_df,
    desc_column='job_description',
    skills_list=skills_all,
    boolean_masks=bool_masks,
    skills_label_dictionary=skills_label_dict,
)

In [20]:
# Creating the dataset for the model

df_labelled_desc = pd.DataFrame()
df_labelled_desc['text'] = input_array
df_labelled_desc['ner_tags'] = label_array

In [21]:
df_labelled_desc

,text,ner_tags
0,"[full, time, ,, in, person, joblocation, :, ne...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1,"[role, :, python, developer, /, professional, ...","[0, 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[jjob, overviewbased, in, guildford, ,, dps, g...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, ..."
3,"[my, client, is, seeking, a, skilled, and, mot...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
4,"[crossover, is, the, world, 's, #, 1, source, ...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...
15296,"[job, title, :, manufacturing, quality, engine...","[0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, ..."
15297,"[the, client, are, a, global, designer, and, m...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, ..."
15298,"[recruit, central, ,, are, currently, on, the,...","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, ..."
15299,"[software, engineer, ,, systematic, equities, ...","[5, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


### 4. Chunking and Tokenising

In [22]:
# Loading hugging face libraries

from transformers import AutoModelForTokenClassification, TrainingArguments\
    , Trainer, AutoTokenizer, DataCollatorForTokenClassification\
    , DataCollatorWithPadding, AutoModelForTokenClassification, pipeline\
    , AutoModelForMaskedLM, default_data_collator

from datasets import ClassLabel, Features, Sequence, Value, Dataset, DatasetDict
from huggingface_hub import notebook_login

import evaluate
import datasets

from sklearn.model_selection import train_test_split




In [23]:

def split_lists_into_chunks(row:pd.Series, n:int, columns:list) -> None:
    # Determine the length of each chunk

    text = row['text']
    labels = row['ner_tags']

    # Split the lists into chunks
    chunks_text = [(text[i:i+n], labels[i:i+n]) for i in range(0, len(text), n)]
    chunks_df = pd.DataFrame(chunks_text, columns=columns)

    return chunks_df



In [24]:

n = 150
df_columns = ['tokens', 'ner_tags']
arrays = []
stop = df_labelled_desc.index[-1]

output = df_labelled_desc.apply(split_lists_into_chunks
                                     , args=(n,df_columns)
                                     , axis=1)




In [25]:
df_tokens = pd.concat([df for df in output], ignore_index=True)

In [26]:
# Setting the features of the dataset object
features = Features(
    {'tokens': Sequence(
        feature=Value(dtype='string')
     ),
     'ner_tags': Sequence(
         feature=ClassLabel(
             num_classes=9,
             names=[
                 'O',
                 'B-BUS',
                 'I-BUS',
                 'B-TECHNOLOGY',
                 'I-TECHNOLOGY',
                 'B-TECHNICAL',
                 'I-TECHNICAL',
                 'B-SOFT',
                 'I-SOFT',
             ],
         )
     )}
)

features

{'tokens': Sequence(feature=Value(dtype='string', id=None), length=-1, id=None),
 'ner_tags': Sequence(feature=ClassLabel(names=['O', 'B-BUS', 'I-BUS', 'B-TECHNOLOGY', 'I-TECHNOLOGY', 'B-TECHNICAL', 'I-TECHNICAL', 'B-SOFT', 'I-SOFT'], id=None), length=-1, id=None)}

In [27]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Convert the pandas DataFrame to a Hugging Face Dataset
train_test_valid_dataset = datasets.Dataset.from_pandas(df_tokens, features=features)


In [28]:
ner_feature = train_test_valid_dataset.features["ner_tags"]
ner_feature

Sequence(feature=ClassLabel(names=['O', 'B-BUS', 'I-BUS', 'B-TECHNOLOGY', 'I-TECHNOLOGY', 'B-TECHNICAL', 'I-TECHNICAL', 'B-SOFT', 'I-SOFT'], id=None), length=-1, id=None)

In [29]:
label_names = ner_feature.feature.names
label_names

['O',
 'B-BUS',
 'I-BUS',
 'B-TECHNOLOGY',
 'I-TECHNOLOGY',
 'B-TECHNICAL',
 'I-TECHNICAL',
 'B-SOFT',
 'I-SOFT']

In [30]:
def align_labels_with_tokens(labels, word_ids):
    new_labels = []
    current_word = None
    for word_id in word_ids:
        if word_id != current_word:
            # Start of a new word!
            current_word = word_id
            label = -100 if word_id is None else labels[word_id]
            new_labels.append(label)
        elif word_id is None:
            # Special token
            new_labels.append(-100)
        else:
            # Same word as previous token
            label = labels[word_id]
            # If the label is B-XXX we change it to I-XXX
            if label % 2 == 1:
                label += 1
            new_labels.append(label)

    return new_labels

In [31]:
def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(
        examples["tokens"], truncation=True, is_split_into_words=True
    )
    all_labels = examples["ner_tags"]
    new_labels = []
    list_word_ids = []
    for i, labels in enumerate(all_labels):
        word_ids = tokenized_inputs.word_ids(i)
        new_labels.append(align_labels_with_tokens(labels, word_ids))
        list_word_ids.append(word_ids)
    tokenized_inputs["labels"] = new_labels
    tokenized_inputs["word_ids"] = list_word_ids

    return tokenized_inputs

In [32]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

# Convert the pandas DataFrame to a Hugging Face Dataset
train_test_valid_dataset = datasets.Dataset.from_pandas(df_tokens, features=features)


In [33]:
# Tokenise and align all the tokens and labels

ner_tokenised_datasets = train_test_valid_dataset.map(
    tokenize_and_align_labels,
    batched=True,
    remove_columns=train_test_valid_dataset.column_names,
)

Map:   0%|          | 0/66571 [00:00<?, ? examples/s]

In [34]:
ner_tokenised_datasets

Dataset({
    features: ['input_ids', 'attention_mask', 'labels', 'word_ids'],
    num_rows: 66571
})

In [35]:
# Split trainning/val/test set
# 70% train, 30% test + validation
train_test_valid_dataset = ner_tokenised_datasets.train_test_split(test_size=0.3)

# Split the 30% test + valid in half test, half valid
test_valid_dataset = train_test_valid_dataset['test'].train_test_split(test_size=0.5)

# Organise to have a single DatasetDict
ner_tokenised_datasets = DatasetDict({
    'train': train_test_valid_dataset['train'],
    'test': test_valid_dataset['test'],
    'valid': test_valid_dataset['train']})

In [36]:
ner_tokenised_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 46599
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 9986
    })
    valid: Dataset({
        features: ['input_ids', 'attention_mask', 'labels', 'word_ids'],
        num_rows: 9986
    })
})

---

### LM Model Training

In [37]:
import evaluate


metric = evaluate.load("seqeval")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)

    # Remove ignored index (special tokens) and convert to labels
    true_labels = [[label_names[l] for l in label if l != -100] for label in labels]
    true_predictions = [
        [label_names[p] for (p, l) in zip(prediction, label) if l != -100]
        for prediction, label in zip(predictions, labels)
    ]
    all_metrics = metric.compute(predictions=true_predictions, references=true_labels)
    return {
        "precision": all_metrics["overall_precision"],
        "recall": all_metrics["overall_recall"],
        "f1": all_metrics["overall_f1"],
        "accuracy": all_metrics["overall_accuracy"],
    }

In [38]:
model_checkpoint = "algiraldohe/distilbert-base-uncased-linkedin-domain-adaptation"
model = AutoModelForTokenClassification.from_pretrained(
    model_checkpoint,
    id2label=id2label,
    label2id=label2id,
)

model_name = "lm-ner-linkedin-skills-recognition"

task = "linkedin-domain-adaptation"
batch_size = 64

# Show the training loss with every epoch
logging_steps = len(ner_tokenised_datasets["train"]) // batch_size

# Define the data collator
data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

Some weights of the model checkpoint at algiraldohe/distilbert-base-uncased-linkedin-domain-adaptation were not used when initializing DistilBertForTokenClassification: ['vocab_transform.weight', 'vocab_projector.bias', 'vocab_projector.weight', 'vocab_transform.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertForTokenClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DistilBertForTokenClassification were not initialized from the model checkpoint at algiraldohe/distilbert-base-uncased-linkedin-domain-adaptation and are newly ini

In [39]:
from huggingface_hub import notebook_login

notebook_login()

In [40]:
training_args = TrainingArguments(
    output_dir=model_name,
    overwrite_output_dir=True,
    evaluation_strategy="epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    num_train_epochs=3,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    push_to_hub=True,
    fp16=True,
    logging_steps=logging_steps,

)


In [41]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=ner_tokenised_datasets["train"],
    eval_dataset=ner_tokenised_datasets["valid"],
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=tokenizer,
)

/content/lm-ner-linkedin-skills-recognition is already a clone of https://huggingface.co/algiraldohe/lm-ner-linkedin-skills-recognition. Make sure you pull the latest changes with `repo.git_pull()`.


In [42]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Precision,Recall,F1,Accuracy
1,0.130100,0.046815,0.878552,0.871502,0.875013,0.986274
2,0.043200,0.034529,0.899405,0.921895,0.910511,0.989974
3,0.033200,0.030732,0.911869,0.931205,0.921436,0.991222


TrainOutput(global_step=2187, training_loss=0.06880346177700289, metrics={'train_runtime': 1300.8609, 'train_samples_per_second': 107.465, 'train_steps_per_second': 1.681, 'total_flos': 7819349405361858.0, 'train_loss': 0.06880346177700289, 'epoch': 3.0})

In [43]:
trainer.push_to_hub(commit_message="Training complete")

Upload file pytorch_model.bin:   0%|          | 1.00/253M [00:00<?, ?B/s]

Upload file runs/Jul07_22-06-49_1e3a323724ab/events.out.tfevents.1688767615.1e3a323724ab.13081.0:   0%|       …

To https://huggingface.co/algiraldohe/lm-ner-linkedin-skills-recognition
   78a0fb5..ec4d1bf  main -> main

   78a0fb5..ec4d1bf  main -> main

To https://huggingface.co/algiraldohe/lm-ner-linkedin-skills-recognition
   ec4d1bf..0549db0  main -> main

   ec4d1bf..0549db0  main -> main



'https://huggingface.co/algiraldohe/lm-ner-linkedin-skills-recognition/commit/ec4d1bf9bad8adec9ff9c3dfb9c09ea631f7eaf4'